In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
from keras.models import load_model
import json

Using TensorFlow backend.


In [20]:
# Check headers of test DICOM images
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']
dcm = pydicom.dcmread(test_dicoms[0])
dcm

(0008, 0016) SOP Class UID                       UI: Secondary Capture Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.3.6.1.4.1.11129.5.5.110503645592756492463169821050252582267888
(0008, 0060) Modality                            CS: 'DX'
(0008, 1030) Study Description                   LO: 'No Finding'
(0010, 0020) Patient ID                          LO: '2'
(0010, 0040) Patient's Sex                       CS: 'M'
(0010, 1010) Patient's Age                       AS: '81'
(0018, 0015) Body Part Examined                  CS: 'CHEST'
(0018, 5100) Patient Position                    CS: 'PA'
(0020, 000d) Study Instance UID                  UI: 1.3.6.1.4.1.11129.5.5.112507010803284478207522016832191866964708
(0020, 000e) Series Instance UID                 UI: 1.3.6.1.4.1.11129.5.5.112630850362182468372440828755218293352329
(0028, 0002) Samples per Pixel                   US: 1
(0028, 0004) Photometric Interpretation          CS: 'MONOCHROME2'
(0028, 0010) Rows       

In [3]:
dcm.BodyPartExamined

'RIBCAGE'

In [52]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    
    # todo
    
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename) 
    img = ds.pixel_array
    if ds.Modality=='DX' and ds.BodyPartExamined=='CHEST'and ds.PatientPosition=='PA':
        return img  
    elif ds.Modality=='DX' and ds.BodyPartExamined=='CHEST'and ds.PatientPosition=='AP':
        return img
    else:
        print('Invalid DICOM')
        return None
    
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean, img_std,img_size): 
    # todo
    img_mean = np.mean(img)
    img_std = np.std(img)
    new_img = img.copy()
    new_img = (new_img - img_mean/img_std)
    img_proc = np.resize(new_img, img_size)
    return img_proc

# This function loads in our trained model w/ weights and compiles it 
#def load_model(model_path, weight_path):
    # todo
    #model_path = model_path
    #model = load_model(weight_path)
    #return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img_proc, thresh): 
    # todo    
    model.load_weights(weight_path)
    pred = model.predict(img_proc)
    if pred > thresh:
        return 'Pneumonia'
    else:
        return 'No Pneumonia'
    #return pred

In [53]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = ('/home/workspace/my_model.json')
weight_path = ('/home/workspace/#11_experiment_my_model.best.hdf5')

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16
#img_mean = np.mean() # loads the mean image value they used during training preprocessing
#img_std = np.std() # loads the std dev image value they used during training preprocessing

my_model_4 = load_model(weight_path) #loads model
thresh = 0.52 #loads the threshold they chose for model classification 

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
    
    img_mean = np.mean(img)
    img_std = np.std(img)   
           
    img_proc = preprocess_image(img,img_mean, img_std,IMG_SIZE)
    pred = predict_image(my_model_4, img_proc, thresh)
    print(pred)

Load file test1.dcm ...
No Pneumonia
Load file test2.dcm ...
No Pneumonia
Load file test3.dcm ...
No Pneumonia
Load file test4.dcm ...
Invalid DICOM
Load file test5.dcm ...
Invalid DICOM
Load file test6.dcm ...
Invalid DICOM
